<a href="https://colab.research.google.com/github/BriniMohamedAyechi/PDF_Q-A_with_LocalLLm_LocalEmbeddings/blob/main/PDF_Q%26A_LocalLLm_LocalEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding faiss-cpu
!pip -q install accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 106.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


#Load the pre trained model and the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large",
                                              load_in_8bit=True,
                                              device_map='auto',
                                              )

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

#Create a pipeline for our local LLM
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLM

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [ ]:
#InstructorEmbedding
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive"

Mounted at /content/gdrive


In [ ]:
#Load the PDF documents
loader = DirectoryLoader(f'{root_dir}/Documents/',glob="./*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
documents

[Document(page_content='City of J onesboro  \n \n Password Policy   Created: 9/27/2017  \n Section of: Corporate Security Policies   Target Audience: Users, Technical  \n CONFIDENTIAL   Page 1 of 4  \n   \nPassword Policy  Confidential  City of J onesboro is hereinafter referred to as "the company."  \n \n1.0 Overview  \n \nA solid password policy is perhaps the most important security control an organization can \nemploy.  Since the responsibility for choosing good passwords falls on the users, a detailed and \neasy-to-understand policy is essential.  \n \n2.0 Purpose  \n \nThe purpose of this policy is to specify guidelines for use of passwords.  Most importantly, this \npolicy will help users understand why strong passwords are a necessity, and help them create \npasswords that are both secure and useable.  Lastly, this policy will educate users on the secure \nuse of passwords.  \n \n3.0 Scope  \n \nThis policy applies to any per son who is provided an account on the organization\'

In [ ]:
#split documents into  chunks of text
text_splitter = RecursiveCharacterTextSplitter( chunk_size = 1000,chunk_overlap=200)

texts = text_splitter.split_documents(documents)

In [ ]:
texts[0]

Document(page_content='City of J onesboro  \n \n Password Policy   Created: 9/27/2017  \n Section of: Corporate Security Policies   Target Audience: Users, Technical  \n CONFIDENTIAL   Page 1 of 4  \n   \nPassword Policy  Confidential  City of J onesboro is hereinafter referred to as "the company."  \n \n1.0 Overview  \n \nA solid password policy is perhaps the most important security control an organization can \nemploy.  Since the responsibility for choosing good passwords falls on the users, a detailed and \neasy-to-understand policy is essential.  \n \n2.0 Purpose  \n \nThe purpose of this policy is to specify guidelines for use of passwords.  Most importantly, this \npolicy will help users understand why strong passwords are a necessity, and help them create \npasswords that are both secure and useable.  Lastly, this policy will educate users on the secure \nuse of passwords.  \n \n3.0 Scope  \n \nThis policy applies to any per son who is provided an account on the organization\'s

In [ ]:
len(texts)

9

In [ ]:
#load the Local embeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name ="intfloat/e5-large-v2")

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
#Store the texts in the semantic search Vector FAISS
import pickle
import faiss
from langchain.vectorstores import FAISS
db_insturctEmbedd = FAISS.from_documents(texts,instructor_embeddings)

In [ ]:
retriever = db_insturctEmbedd.as_retriever(search_kwargs={"k":3})

In [ ]:
retriever.search_type

'similarity'

In [ ]:
docs = retriever.get_relevant_documents("does this document contain password complexity?")

In [ ]:
docs[0]

Document(page_content='of organization passwords:  \n \n• Users must not disclose their passwords to anyone  \n \n• Users must not share their passwords with others (co- workers, supervisors, family, etc.)  \n \n• Users must not write down their passwords and leave them unsecured  \n \n• Users must not check the "save password" box when authenticating to applications  \n \n• Users must not use the same password for different systems and/or accounts  \n \n• Users must not send passwords via email', metadata={'source': '/content/gdrive/My Drive/Documents/password_policy_201806040830134338.pdf', 'page': 1})

In [ ]:
type(docs)

list

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
#test your model using the Query
query = 'does this document contain password complexity?'

print('-------------------Instructor Embeddings------------------\n')
llm_response = qa_chain(query)
process_llm_response(llm_response)